In [11]:
from pytradfri import Gateway
from pytradfri.api.libcoap_api import APIFactory
from pytradfri.util import load_json, save_json
from pytradfri.error import PytradfriError
from pytradfri.command import Command
from pytradfri.device.light import Light
import uuid
from typing import List
from pytradfri.const import LIGHT_DIMMER

In [12]:

GATEWAY_IP = "192.168.1.175"

CONFIG_FILE = "tradfri_standalone_psk.conf"

In [13]:
APIFactory(GATEWAY_IP).psk

In [14]:
if not "":
    print("AAA")

AAA


In [15]:
def signe_up(ip, key):
    
    identity = uuid.uuid4().hex
    api_factory =  APIFactory(host=ip, psk_id=identity)

    psk = api_factory.generate_psk(key)

    return identity, psk
identity, psk = signe_up(GATEWAY_IP, GATEWAY_KEY)
identity, psk

('1f80c7a847714ece963e980a13d90abe', 'twjcMTPV7vovjhzC')

In [16]:
light_ids = [
  65540, 65541, 65542,
  65543, 65544, 65545,
  65546, 65547, 65553,
  65552, 65551, 65550,
  65549
]

In [17]:
import warnings
warnings.filterwarnings("ignore", message="Field \"model_number\" has conflict with protected namespace \"model_\".")


In [18]:
api_factory = APIFactory(host=GATEWAY_IP, psk_id=identity, psk=psk, timeout=10)
api = api_factory.request
gateway = Gateway()

In [19]:
gateway_info = api(Gateway.get_gateway_info())

# Print the firmware version
print("Gateway Firmware Version:", gateway_info.firmware_version)

Gateway Firmware Version: 1.21.31


In [20]:
devices_command = gateway.get_devices()
print(devices_command)
devices_commands: List[Command] = api(devices_command)
devices_commands = [dc for dc in devices_commands if light_ids.__contains__(dc.path[-1])]

devices_commands[0].path[-1] = str(devices_commands[0].path[-1])
device = api(devices_commands[0])

print(device)

<Command get ['15001']>
<65541 - TRADFRI bulb 2 (TRADFRI bulb E14 CWS 470lm)>
